# Cleaning Data

### Import ✔️

In [1]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')
mne.set_log_level('warning') #All mne functions will only spit out output if they have reason to.


### Input Data ✔️

In [2]:
epochs = mne.read_epochs(pathlib.Path('out_data')
            / 'Epochs' / 'sub-01_ses-01_task-WM1isi0_epochs-epo.fif')

### Reject Bad Epochs ❌

In [3]:
reject_criteria = dict(eeg=100e-6) # 100 µV
#flat_criteria = dict(eeg=1e-6)           # 1 µV

epochs.drop_bad(reject=reject_criteria)
#epochs.drop_bad(reject=reject_criteria, flat=flat_criteria)

Number of events,260
Events,TsD-: 55WMD+: 75bgin: 57fix+: 73
Time range,-0.250 – 0.800 sec
Baseline,-0.250 – 0.000 sec


In [4]:
epochs.plot()

### ICA Analysis ❌

Load in a new series of raw data, and pass a 1.0 Hz high-pass filter.

In [5]:
bids_root = pathlib.Path('out_data/BIDS_Structure')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='WM1isi0',
                              datatype='eeg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
#Apply a new filter
raw.filter(l_freq=1, h_freq=40)

C:\Users\trevo\AppData\Local\Temp\ipykernel_9108\3004586996.py:9: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['SESS', 'CELL', 'bgin', 'fix+', 'WMD+', 'TsD-', 'TRSP', 'STI 014']
  raw = mne_bids.read_raw_bids(bids_path)
C:\Users\trevo\AppData\Local\Temp\ipykernel_9108\3004586996.py:9: RuntimeWarning: Not setting positions of 8 stim channels found in montage:
['SESS', 'CELL', 'bgin', 'fix+', 'WMD+', 'TsD-', 'TRSP', 'STI 014']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne_bids.read_raw_bids(bids_path)


<RawBrainVision | sub-01_ses-01_task-WM1isi0_eeg.eeg, 137 x 361490 (361.5 s), ~378.0 MB, data loaded>

In [6]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'Epochs' / 
                         'sub-01_ses-01_task-WM1isi0_epochs-epo.fif')
#epochs.selection is an array of all epochs that were not thrown out.
epochs_selection = epochs.selection
epochs_selection

array([  9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,
        22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,
        35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,
        48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
        87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
       100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
       152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
       165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
       178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 18

In [7]:
events, event_id = mne.events_from_annotations(raw)
events = events[epochs_selection]

In [8]:
#Filtering to remove slow drifts
filt_raw = raw.copy().filter(l_freq=1., h_freq=None)
filt_raw

<RawBrainVision | sub-01_ses-01_task-WM1isi0_eeg.eeg, 137 x 361490 (361.5 s), ~378.0 MB, data loaded>

In [9]:
tmin = epochs.tmin #-.250
tmax = epochs.tmax #.800

#Fit the epochs election into this epoch constructor
epochs_ica = mne.Epochs(raw,
                        events=events,
                        event_id=event_id,
                        tmin=tmin,
                        tmax=tmax,
                        preload=True)

In [10]:
epochs_ica.info

<Info | 12 non-empty values
 bads: []
 ch_names: E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11, E12, E13, E14, ...
 chs: 129 EEG, 8 Stimulus
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 129 items (129 EEG)
 experimenter: mne_anonymize
 highpass: 1.0 Hz
 line_freq: 60.0
 lowpass: 40.0 Hz
 meas_date: 2019-11-10 20:00:11 UTC
 nchan: 137
 projs: []
 sfreq: 1000.0 Hz
 subject_info: 4 items (dict)
>

In [11]:
n_components = 0.999  

method = 'picard' 
max_iter = 1000  
fit_params = dict(fastica_it=5)
random_state = 42 

#Actually runs the ICA
ica = mne.preprocessing.ICA(n_components=n_components,
                            #max_pca_components=300,
                            method=method,
                            max_iter=max_iter,
                            fit_params=fit_params,
                            random_state=random_state)
ica.fit(epochs_ica)

C:\Users\trevo\AppData\Local\Temp\ipykernel_9108\1746337714.py:15: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs_ica)


Method,picard
Fit,51 iterations on epochs (528653 samples)
ICA components,25
Explained variance,99.9 %
Available PCA components,129
Channel types,eeg
ICA components marked for exclusion,—


In [ ]:
ica.plot_components(inst=epochs)

In [ ]:
epochs_cleaned = ica.apply(epochs.copy()) 

In [ ]:
#Ploting both after and before plots of epoched data
epochs_cleaned.plot(title='After ICA')
epochs.plot(title='Before ICA')